# **LP1_Run2**

In [ ]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 39.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 7.2 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [ ]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [ ]:
problem = """A firm from Milan sells chemical products for professional cosmetics. It is planning the production of three products, GCA, GCB and GCC, for a given period of
time by mixing two different components: C1 and C2. All the end products must
contain at least one of the two components, and not necessarily both.

For the next planning period, 10,000 l of C1 and 15,000 l of C2 are available.
The production of GCA, GCB and GCC must be scheduled to at least cover the
minimum demand level of 6,000, 7,000 and 9,000 l, respectively. It is assumed
that when chemical components are mixed, there is no loss or gain in volume.

Each chemical component, C1 and C2, has a proportional critical element, 0.4
and 0.2, respectively. That is to say, each litre of C1 contains 0.4 l of the critical
element. To obtain GCA, the mixture must proportionally contain at least a 0.3
fraction of the critical element. Another requirement is that the quantity of the
critical element is seen in GCB, a 0.3 fraction at the most.
Furthermore, the minimum ratio of C1 with C2 in product GCC must be 0.3.

The profit expected for the sale of each litre of GCA, GCB and GCC is $120, $135
and $155, respectively.

Optimise the production planning of this firm.  """

In [ ]:


response1 = openai.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 2
)


In [ ]:
print(response1.choices[0].message.content)

Parameters:

Ci=volume of component i available  for i=1,2. C1= 10,000 l and C2=15,000 l

Di=Minimum demand for product i for i= A, B, C. DA=6,000 l, DB=7,000 l, DC=9,000 l

Ei=proportional critical element in component i for i=1,2. E1=0.4 and E2=0.2

Fi=proportional fraction of critical element in product i for i=A,B. FA=0.3 and FB=0.3

R=minimum ratio of C1 with C2 in GCC, R=0.3

Pi=profit per litre of product i for i=A,B,C. PA=$120, PB=$135, PC=$155

Decision Variables:

Xij=volume of component i used in product j, for i=1,2 and j=A,B,C

Objective Function:

Maximize the total profit Z=PA*(X1A+X2A)+PB*(X1B+X2B)+PC*(X1C+X2C)

Subject to Constraints:

For meeting the demand,
X1A+X2A >= DA
X1B+X2B >= DB
X1C+X2C >= DC

For the availability of components,
X1A+X1B+X1C <= C1
X2A+X2B+X2C <= C2

For the critical elements in products,
E1*X1A+E2*X2A >= FA*(X1A+X2A)
E1*X1B+E2*X2B <= FB*(X1B+X2B)

For the minimum ratio in GCC,
X1C/(X1C+X2C) >= R

All decision variables are non-negative, 
Xij>=0 

### **Generate Pyomo Code**

In [ ]:

response2 = openai.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [ ]:
Markdown(response2.choices[0].message.content)

Below is the python Pyomo code for your optimization problem:

```python
import pyomo.environ as pyEnv

model = pyEnv.ConcreteModel()

# Declaration of indices
model.i = pyEnv.Set(initialize=['1','2']) #Index for Component
model.j = pyEnv.Set(initialize=['A','B','C']) #Index for Product

# Declaration of Parameters
V_c = {'1': 10000, '2': 15000}   # Sample Data : [Component 1, Component 2] volume
D = {'A': 6000, 'B': 7000, 'C': 9000}  # Sample Data : Demand of products [A, B, C]
E = {'1': 0.4, '2': 0.2}  # Sample Data: proportional critical element in Component [1, 2]
F = {'A': 0.3, 'B': 0.3} # Sample Data: proportional fraction of critical element in product [A, B]
P = {'A': 120, 'B': 135, 'C': 155} # Sample Data: Profit per litre of Product [A, B, C]
R = 0.3  # Sample Data: minimum ratio of Component 1 with Component 2 in GCC

# Declaration of Decision Variables
model.X = pyEnv.Var(model.i,model.j ,domain=pyEnv.NonNegativeReals)

# Objective Function
def Zrule(model):
    profit = sum(P[j]*(model.X['1',j]+model.X['2',j]) for j in model.j)
    return profit
model.Z=pyEnv.Objective(rule=Zrule, sense=pyEnv.maximize)

# Constraints
def demandRule(model, j):
    return sum(model.X[i,j] for i in model.i) >= D[j]
model.demandConstr = pyEnv.Constraint(model.j, rule=demandRule)

def volumeRule(model, i):
    return sum(model.X[i,j] for j in model.j) <= V_c[i]
model.volumeConstr = pyEnv.Constraint(model.i, rule=volumeRule)

def criticalElmARule(model):
    return E['1']*model.X['1','A']+E['2']*model.X['2','A'] >= F['A'] * (model.X['1','A'] + model.X['2','A'])
model.criticalElmAConstr = pyEnv.Constraint(rule=criticalElmARule)

def criticalElmBRule(model):
    return E['1']*model.X['1','B']+E['2']*model.X['2','B'] <= F['B'] * (model.X['1','B'] + model.X['2','B'])
model.criticalElmBConstr = pyEnv.Constraint(rule=criticalElmBRule)

def minimumRatioRule(model):
    return model.X['1','C']/(model.X['1','C']+model.X['2','C']) >= R
model.minimumRatioConstr = pyEnv.Constraint(rule=minimumRatioRule)
```

In the above code, we first import required libraries and then initialized set of indices, parameters and decision variables. The objective function and constraints are then defined with help of these declarations. Note that, the values used in the parameters are just some sample data and should be replaced with actual given data.

### **Run the code Generated by GPT4**

In [ ]:
import pyomo.environ as pyEnv

model = pyEnv.ConcreteModel()

# Declaration of indices
model.i = pyEnv.Set(initialize=['1','2']) #Index for Component
model.j = pyEnv.Set(initialize=['A','B','C']) #Index for Product

# Declaration of Parameters
V_c = {'1': 10000, '2': 15000}   # Sample Data : [Component 1, Component 2] volume
D = {'A': 6000, 'B': 7000, 'C': 9000}  # Sample Data : Demand of products [A, B, C]
E = {'1': 0.4, '2': 0.2}  # Sample Data: proportional critical element in Component [1, 2]
F = {'A': 0.3, 'B': 0.3} # Sample Data: proportional fraction of critical element in product [A, B]
P = {'A': 120, 'B': 135, 'C': 155} # Sample Data: Profit per litre of Product [A, B, C]
R = 0.3  # Sample Data: minimum ratio of Component 1 with Component 2 in GCC

# Declaration of Decision Variables
model.X = pyEnv.Var(model.i,model.j ,domain=pyEnv.NonNegativeReals)

# Objective Function
def Zrule(model):
    profit = sum(P[j]*(model.X['1',j]+model.X['2',j]) for j in model.j)
    return profit
model.Z=pyEnv.Objective(rule=Zrule, sense=pyEnv.maximize)

# Constraints
def demandRule(model, j):
    return sum(model.X[i,j] for i in model.i) >= D[j]
model.demandConstr = pyEnv.Constraint(model.j, rule=demandRule)

def volumeRule(model, i):
    return sum(model.X[i,j] for j in model.j) <= V_c[i]
model.volumeConstr = pyEnv.Constraint(model.i, rule=volumeRule)

def criticalElmARule(model):
    return E['1']*model.X['1','A']+E['2']*model.X['2','A'] >= F['A'] * (model.X['1','A'] + model.X['2','A'])
model.criticalElmAConstr = pyEnv.Constraint(rule=criticalElmARule)

def criticalElmBRule(model):
    return E['1']*model.X['1','B']+E['2']*model.X['2','B'] <= F['B'] * (model.X['1','B'] + model.X['2','B'])
model.criticalElmBConstr = pyEnv.Constraint(rule=criticalElmBRule)

def minimumRatioRule(model):
    return model.X['1','C']/(model.X['1','C']+model.X['2','C']) >= R
model.minimumRatioConstr = pyEnv.Constraint(rule=minimumRatioRule)

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [ ]:
import pyomo.environ as pyEnv
from pyomo.environ import *
model = pyEnv.ConcreteModel()

# Declaration of indices
model.i = pyEnv.Set(initialize=['1','2']) #Index for Component
model.j = pyEnv.Set(initialize=['A','B','C']) #Index for Product

# Declaration of Parameters
V_c = {'1': 10000, '2': 15000}   # Sample Data : [Component 1, Component 2] volume
D = {'A': 6000, 'B': 7000, 'C': 9000}  # Sample Data : Demand of products [A, B, C]
E = {'1': 0.4, '2': 0.2}  # Sample Data: proportional critical element in Component [1, 2]
F = {'A': 0.3, 'B': 0.3} # Sample Data: proportional fraction of critical element in product [A, B]
P = {'A': 120, 'B': 135, 'C': 155} # Sample Data: Profit per litre of Product [A, B, C]
R = 0.3  # Sample Data: minimum ratio of Component 1 with Component 2 in GCC

# Declaration of Decision Variables
model.X = pyEnv.Var(model.i,model.j ,domain=pyEnv.NonNegativeReals)

# Objective Function
def Zrule(model):
    profit = sum(P[j]*(model.X['1',j]+model.X['2',j]) for j in model.j)
    return profit
model.Z=pyEnv.Objective(rule=Zrule, sense=pyEnv.maximize)

# Constraints
def demandRule(model, j):
    return sum(model.X[i,j] for i in model.i) >= D[j]
model.demandConstr = pyEnv.Constraint(model.j, rule=demandRule)

def volumeRule(model, i):
    return sum(model.X[i,j] for j in model.j) <= V_c[i]
model.volumeConstr = pyEnv.Constraint(model.i, rule=volumeRule)

def criticalElmARule(model):
    return E['1']*model.X['1','A']+E['2']*model.X['2','A'] >= F['A'] * (model.X['1','A'] + model.X['2','A'])
model.criticalElmAConstr = pyEnv.Constraint(rule=criticalElmARule)

def criticalElmBRule(model):
    return E['1']*model.X['1','B']+E['2']*model.X['2','B'] <= F['B'] * (model.X['1','B'] + model.X['2','B'])
model.criticalElmBConstr = pyEnv.Constraint(rule=criticalElmBRule)

def minimumRatioRule(model):
    return model.X['1','C'] >= R*(model.X['1','C']+model.X['2','C'])
model.minimumRatioConstr = pyEnv.Constraint(rule=minimumRatioRule)

SolverFactory('glpk').solve(model)

# Print the optimal solution
print("Optimal Solution:")
print(f"Total Profit = {model.Z():.2f}")
for j in model.j:
    print(f"Product {j}:")
    for i in model.i:
        print(f"  Component {i}: {model.X[i, j]():.2f} liters")



Optimal Solution:
Total Profit = 3525000.00
Product A:
  Component 1: 3000.00 liters
  Component 2: 3000.00 liters
Product B:
  Component 1: 3400.00 liters
  Component 2: 3600.00 liters
Product C:
  Component 1: 3600.00 liters
  Component 2: 8400.00 liters
